In [2]:
import pandas as pd

In [3]:
# Пути к данным
house_path = "/content/data/house_dataset.csv"
antagonists_path = "/content/data/antagonists_dataset.csv"

# Загрузка данных
house_df = pd.read_csv(house_path)
antagonists_df = pd.read_csv(antagonists_path)

# Вывод первых строк для проверки
print("Первые строки датасета Доктора Хауса:")
print(house_df.head())

print("Первые строки датасета антагонистов:")
print(antagonists_df.head())

Первые строки датасета Доктора Хауса:
    name  season                                      previous_line  \
0  house       1  Fair enough. I dont like healthy Patients. The...   
1  house       1  Shouldnt we be speaking to the Patient before ...   
2  house       1                                           No, but!   
3  house       1      Isnt treating Patients why we became doctors?   
4  house       1                                           Mad cow?   

  previous_speaker                                       context_long  \
0           wilson  See that? They all assume Im a Patient because...   
1          foreman  And shes not responding to radiation treatment...   
2          foreman  Come on! Why leave all the fun for the coroner...   
3          foreman  Shouldnt we be speaking to the Patient before ...   
4            chase  First year of medical school if you hear hoof ...   

   is_greeting  is_question  is_negation  is_exclamation  is_sarcasm  \
0        False        Fa

**Отличия очистки данных для кросс-энкодера**

Очистка данных для кросс-энкодера отличается от стандартной предобработки, так как модель оценивает соответствие пары "контекст + вопрос → ответ". Основные различия:

1. Контекст должен оставаться информативным. Не удаляем вопросы из контекста (previous_line, context_1, context_2 и т. д.), потому что они важны для понимания диалога.
Вопрос из контекста помогает модели различать правильные и неправильные ответы.

2.  Негативные примеры (антагонисты) должны быть осмысленными. Оставляем все антагонистические реплики, но удаляем вопросы, чтобы не запутывать модель. Негативные примеры должны быть похожи на реальные ответы, но не соответствовать контексту.
3. Ограничение длины контекста. BERT-кросс-энкодер имеет ограничение 512 токенов, поэтому: оставляем только 2-3 предыдущие реплики перед вопросом (context_1 – context_3);
ограничиваем контекст до 100 слов (больше, чем для биэнкодера), чтобы не перегружать модель. Ограничиваем ответы до 40 слов, чтобы избежать обрезания важных частей.
4. Фильтрация коротких реплик. Для Доктора Хауса фильтруем реплики короче 3 слов. Для антагонистов оставляем вообще все реплики, кроме вопросов.

In [5]:
import re
import pandas as pd

# --- ФУНКЦИИ ОЧИСТКИ ---
def clean_text(text):
    """Очистка текста от проблемных символов, лишних пробелов и мусора."""
    if not isinstance(text, str) or len(text) == 0:
        return ""

    text = text.replace("�", "")  # Удаление неизвестных символов
    text = re.sub(r"\s+", " ", text).strip()  # Убираем лишние пробелы
    text = re.sub(r"[^\x00-\x7F]+", "", text)  # Удаляем любые не-ASCII символы (если есть)

    return text

def additional_cleaning(text):
    """Дополнительная очистка с учетом чисел, процентов, дробей и случайных заглавных букв."""
    if not isinstance(text, str):
        return text

    text = re.sub(r"[^\w\s.,!?'$/%-]", "", text)  # Разрешенные символы
    text = re.sub(r"\s+", " ", text).strip()  # Удаление лишних пробелов

    return text

def apply_text_cleaning(df, text_columns):
    """Очистка всех указанных текстовых колонок."""
    for col in text_columns:
        df[col] = df[col].astype(str).apply(clean_text).apply(additional_cleaning)
    return df

# --- ФИЛЬТРАЦИЯ ДАННЫХ ---
def count_words(text):
    """Подсчет количества слов в тексте."""
    return len(text.split()) if isinstance(text, str) else 0

def filter_dataframe(df, min_word_count, remove_questions=True, text_column="line"):
    """Фильтрация реплик: удаление коротких, дубликатов, вопросов и лишних символов."""

    initial_count = len(df)

    # Удаление пустых значений
    df = df.dropna(subset=[text_column]).reset_index(drop=True)

    # Удаление слишком коротких реплик (<10 символов)
    df = df[df[text_column].str.len() >= 10].reset_index(drop=True)

    # Удаление вопросов, если это колонка с ответами (`line`)
    if remove_questions:
        df = df[~df[text_column].str.endswith("?")].reset_index(drop=True)

    # Удаление дубликатов
    df = df.drop_duplicates(subset=[text_column]).reset_index(drop=True)

    # Фильтрация реплик по количеству слов
    df["word_count"] = df[text_column].apply(count_words)
    df = df[df["word_count"] >= min_word_count].drop(columns=["word_count"]).reset_index(drop=True)

    return df

# --- СОКРАЩЕНИЕ КОНТЕКСТА ---
def get_shortened_context(row, max_replies=3):
    """Формирует контекст, ограниченный 2-3 репликами перед вопросом."""
    context_parts = []

    for i in range(1, max_replies + 1):  # Оставляем context_1, context_2, context_3
        if pd.notna(row[f"context_{i}"]):
            context_parts.append(row[f"context_{i}"])

    return " [SEP] ".join(context_parts)

def truncate_text(text, max_words):
    """Обрезает текст до указанного количества слов."""
    words = text.split()
    return " ".join(words[:max_words]) if len(words) > max_words else text

# --- ОЧИСТКА И ФИЛЬТРАЦИЯ ---
# Очистка текстовых колонок
text_columns_house = ["line", "previous_line", "context_1", "context_2", "context_3", "context_4", "context_5", "context_long"]
text_columns_antagonists = ["line"]

house_df = apply_text_cleaning(house_df, text_columns_house)
antagonists_df = apply_text_cleaning(antagonists_df, text_columns_antagonists)

# Фильтрация данных
house_df = filter_dataframe(house_df, min_word_count=3, remove_questions=True)
antagonists_df = filter_dataframe(antagonists_df, min_word_count=1, remove_questions=True)

# Генерация укороченного контекста
house_df["short_context"] = house_df.apply(get_shortened_context, axis=1)

# Обрезаем слишком длинные реплики
house_df["line"] = house_df["line"].apply(lambda x: truncate_text(x, 40))
house_df["short_context"] = house_df["short_context"].apply(lambda x: truncate_text(x, 100))

# Итоговая статистика
print(f"\nКоличество реплик после очистки: {len(house_df)} (Доктор Хаус), {len(antagonists_df)} (Антагонисты).")



Количество реплик после очистки: 7417 (Доктор Хаус), 76386 (Антагонисты).


In [6]:
 import re

def is_question(text):
    """Определяет, содержит ли реплика вопрос"""
    if not isinstance(text, str):
        return False

    # Проверка, заканчивается ли текст на вопросительный знак
    if text.strip().endswith("?"):
        return True

    # Поиск вопросительных слов в тексте
    question_words = {"what", "why", "how", "who", "when", "where", "which",
                      "что", "почему", "как", "кто", "когда", "где", "какой", "зачем"}

    words = set(text.lower().split())
    if question_words.intersection(words):
        return True

    # Проверка наличия вопросительного знака внутри реплики
    if "?" in text:
        return True

    # Проверка восклицательных предложений, содержащих вопросительные слова
    if text.endswith("!") and question_words.intersection(words):
        return True

    return False

def analyze_questions(df, context_columns):
    """Подсчитывает количество вопросов в указанных колонках"""
    question_stats = {col: df[col].apply(is_question).sum() for col in context_columns}
    return question_stats

# Определение колонок для анализа
context_columns = ["previous_line", "context_1", "context_2", "context_3", "context_4", "context_5"]

# Анализ вопросов в контексте
question_stats = analyze_questions(house_df, context_columns)

# Вывод статистики по вопросам
print("\nКоличество вопросов в предыдущих репликах:")
for col, count in question_stats.items():
    total = len(house_df)
    print(f"{col}: {count} ({count / total * 100:.2f}%)")

# Выборка примеров реплик с вопросами
print("\nПримеры вопросов в предыдущих репликах:")
for col in context_columns:
    sample_questions = house_df[house_df[col].apply(is_question)][col].dropna().sample(5, random_state=42).tolist()
    print(f"\n{col}:")
    for q in sample_questions:
        print(f"- {q}")



Количество вопросов в предыдущих репликах:
previous_line: 5802 (78.23%)
context_1: 4352 (58.68%)
context_2: 5324 (71.78%)
context_3: 5976 (80.57%)
context_4: 6423 (86.60%)
context_5: 6709 (90.45%)

Примеры вопросов в предыдущих репликах:

previous_line:
- It doesnt matter. I clearly didnt lead him along or anything like that, which proves Im not a tease. So why is your girlfriend mad at you?
- So you just show up Every time hes at physio?
- Youre cutting him open?
- And why are you carrying a vial of it around with you?
- Youre talking about me?

context_1:
- Hes my ex, I Youve got an opinion, too?
- Im 4 1. Thats 1.5 canes in metric. Compared to you Im sure he was huge. Did he have a fetish or did he just fall in love with your longlegged soul?
- And you beliEve him? I understand youre a big fan. Ill have my guy send over a signed glossy.
- Thats not a speech. A few things I forgot to mention. Ed Vogler is a brilliant businessMan. A brilliant Judge of people, and a Man who has nEver 

In [7]:
def count_questions(text):
    """Определяет количество вопросов в реплике"""
    if not isinstance(text, str):
        return 0

    # Разбиваем текст на предложения
    sentences = re.split(r"(?<=[?.!])\s+", text)

    # Счетчик вопросов
    question_count = sum(1 for s in sentences if is_question(s))

    return question_count

# Добавляем колонку с количеством вопросов в предыдущей реплике
house_df["question_count_previous"] = house_df["previous_line"].apply(count_questions)

# Анализ распределения
question_distribution = house_df["question_count_previous"].value_counts().sort_index()

# Вывод распределения количества вопросов в предыдущей реплике
print("\nРаспределение количества вопросов в предыдущей реплике:")
for num_questions, count in question_distribution.items():
    print(f"{num_questions} вопросов: {count} реплик ({count / len(house_df) * 100:.2f}%)")



Распределение количества вопросов в предыдущей реплике:
0 вопросов: 1615 реплик (21.77%)
1 вопросов: 5084 реплик (68.55%)
2 вопросов: 617 реплик (8.32%)
3 вопросов: 83 реплик (1.12%)
4 вопросов: 13 реплик (0.18%)
5 вопросов: 3 реплик (0.04%)
6 вопросов: 1 реплик (0.01%)
8 вопросов: 1 реплик (0.01%)


Выводы из распределения количества вопросов в предыдущей реплике
1. Большинство реплик содержит ровно один вопрос (68.55%), то есть основной контекст перед ответом — это одиночный вопрос.
Вероятно, в диалоге чаще всего звучит один явный вопрос, на который отвечает Доктор Хаус. Включение такого контекста в кросс-энкодер важно, потому что модель должна понимать, что основной сигнал для ответа — это вопрос.
2. 21.77% реплик вообще не содержат вопросов. Иногда ответы следуют после утверждений или размышлений, а не после вопросов.
Нужно не исключать такие реплики, но учитывать, что иногда контекст — не явный вопрос. Для кросс-энкодера это означает, что иногда предыдущая реплика — это не вопрос, а комментарий.
3. 8.32% реплик содержат два вопроса. В таких случаях возможно два варианта: а) Вопросы идут подряд, например: "Ты серьезно? Почему?" б) Вопрос встроен в реплику: "Ты уверен в этом? Я просто не понимаю, как это может работать." Для кросс-энкодера это означает, что иногда нужно учитывать оба вопроса, а не только последний.
4. Редкие случаи с 3+ вопросами (1.35%) Такие реплики встречаются редко, поэтому не стоит делать на них акцент.
Однако, если в реплике 3+ вопросов, скорее всего, важен последний.


In [8]:
def get_text_length(text):
    """Возвращает количество слов в тексте"""
    return len(text.split()) if isinstance(text, str) else 0

# Добавление колонок с длиной текста
length_columns = ["previous_line", "context_1", "context_2", "context_3", "context_4", "context_5", "context_long"]
for col in length_columns:
    house_df[f"{col}_length"] = house_df[col].apply(get_text_length)

# Анализ распределения длины текста
length_stats = house_df[[f"{col}_length" for col in length_columns]].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95])

# Вывод статистики по длине реплик
print("\nРаспределение длины предыдущих реплик и контекста:")
print(length_stats)



Распределение длины предыдущих реплик и контекста:
       previous_line_length  context_1_length  context_2_length  \
count            7417.00000       7417.000000       7417.000000   
mean               10.14251         28.756775         39.188756   
std                 9.81743         20.133525         22.890445   
min                 1.00000          3.000000          5.000000   
25%                 4.00000         15.000000         24.000000   
50%                 7.00000         23.000000         34.000000   
75%                13.00000         36.000000         49.000000   
90%                21.00000         53.000000         67.000000   
95%                28.00000         65.000000         82.000000   
max               152.00000        192.000000        202.000000   

       context_3_length  context_4_length  context_5_length  \
count       7417.000000       7417.000000       7417.000000   
mean          49.940812         60.336524         70.927734   
std           25.5826

1. Предыдущая реплика — в среднем 10 слов, но иногда до 152 слов. В 50% случаев содержит 7 слов или меньше. В 90% случаев не превышает 21 слово.
Вывод: previous_line можно включать целиком в контекст, но если он длиннее 25 слов, стоит обрезать.
2. context_1 — в среднем 28 слов, но может достигать 192 слов
Вывод: context_1 можно включать, но если он длиннее 50 слов, обрезать.
3. context_2 и далее — контекст быстро растет. context_2 в среднем 39 слов (до 202 слов). context_3 в среднем 49 слов (до 218 слов). context_4, context_5 уже 60+ слов.
context_long (все предыдущие реплики) в среднем 77 слов (до 262 слов).
Вывод: context_3+ включать только при необходимости, иначе он будет слишком длинным.

**Решение по формированию контекста.**
- Берем previous_line полностью, если ≤25 слов, иначе обрезаем.
- Берем context_1, если в нем есть вопрос, иначе context_2.
- Обрезаем context_1 и context_2, если они >50 слов.
context_3+ включать только если суммарная длина < 100 слов.


In [9]:
def truncate_text(text, max_words):
    """Обрезает текст до указанного количества слов."""
    words = text.split()
    return " ".join(words[:max_words]) if len(words) > max_words else text

def select_best_context(row):
    """Формирует контекст на основе длины и наличия вопросов"""

    # Обрезка предыдущей реплики, если слишком длинная
    previous = truncate_text(row["previous_line"], 25)

    # Поиск первого контекста с вопросом
    selected_context = []
    for i in range(1, 6):  # context_1 - context_5
        if is_question(row[f"context_{i}"]):  # Если есть вопрос, добавляем
            selected_context.append(truncate_text(row[f"context_{i}"], 50))
            break

    # Если вопросов в контексте нет, берем context_1
    if not selected_context and pd.notna(row["context_1"]):
        selected_context.append(truncate_text(row["context_1"], 50))

    # Если контекст не превышает 100 слов, добавляем context_2
    total_length = sum(len(c.split()) for c in selected_context)
    if total_length < 100 and pd.notna(row["context_2"]):
        selected_context.append(truncate_text(row["context_2"], 50))

    # Финальный контекст
    final_context = " [SEP] ".join([previous] + selected_context)
    return final_context

# Генерация финального контекста
house_df["final_context"] = house_df.apply(select_best_context, axis=1)

# Проверка примеров
print("\nПримеры сформированных контекстов:")
for i in range(5):
    print(f"\nКонтекст {i+1}:")
    print(house_df["final_context"].iloc[i])



Примеры сформированных контекстов:

Контекст 1:
Fair enough. I dont like healthy Patients. The 29 year old female! [SEP] You see where the administration might have a pRoblem with that attitude. The one who cant talk, I liked that part. [SEP] So put on a white coat like the rest of us. You see where the administration might have a pRoblem with that attitude. The one who cant talk, I liked that part.

Контекст 2:
Shouldnt we be speaking to the Patient before we start diagnosing? [SEP] Its a lesion. Is she a doctor? [SEP] And shes not responding to radiation treatment. Its a lesion. Is she a doctor?

Контекст 3:
Isnt treating Patients why we became doctors? [SEP] No, but! No, treating illnesses is why we became doctors, treating Patients is what makes most doctors miserable. [SEP] Shouldnt we be speaking to the Patient before we start diagnosing? No, but! No, treating illnesses is why we became doctors, treating Patients is what makes most doctors miserable.

Контекст 4:
Wernickies ence

In [10]:
def count_questions_in_text(text):
    """Подсчитывает количество вопросов в тексте"""
    if not isinstance(text, str):
        return 0
    sentences = re.split(r"(?<=[?.!])\s+", text)
    return sum(1 for s in sentences if is_question(s))

# Добавление статистики
house_df["final_context_length"] = house_df["final_context"].apply(get_text_length)
house_df["final_context_question_count"] = house_df["final_context"].apply(count_questions_in_text)

# Анализ, какие контексты чаще всего используются
context_usage = {"context_1": 0, "context_2": 0, "context_3": 0, "context_4": 0, "context_5": 0}

for i in range(1, 6):
    context_usage[f"context_{i}"] = house_df[f"context_{i}"].notna().sum()

# Вывод статистики по длине контекста
length_stats = house_df["final_context_length"].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95])
question_stats = house_df["final_context_question_count"].describe()

print("\nСтатистика по длине сформированного контекста:")
print(length_stats)

print("\nСтатистика по количеству вопросов в контексте:")
print(question_stats)

print("\nЧастота использования разных уровней контекста:")
for key, value in context_usage.items():
    print(f"{key}: {value} раз использован ({value / len(house_df) * 100:.2f}%)")

# Примеры длинных контекстов
print("\nПримеры длинных контекстов:")
long_contexts = house_df[house_df["final_context_length"] > 100]["final_context"].sample(5, random_state=42).tolist()
for i, context in enumerate(long_contexts, 1):
    print(f"\nКонтекст {i}:\n{context}")



Статистика по длине сформированного контекста:
count    7417.000000
mean       76.700822
std        26.311406
min        13.000000
25%        55.000000
50%        77.000000
75%       101.000000
90%       110.000000
95%       115.000000
max       127.000000
Name: final_context_length, dtype: float64

Статистика по количеству вопросов в контексте:
count    7417.000000
mean        3.135095
std         1.674280
min         0.000000
25%         2.000000
50%         3.000000
75%         4.000000
max        12.000000
Name: final_context_question_count, dtype: float64

Частота использования разных уровней контекста:
context_1: 7417 раз использован (100.00%)
context_2: 7417 раз использован (100.00%)
context_3: 7417 раз использован (100.00%)
context_4: 7417 раз использован (100.00%)
context_5: 7417 раз использован (100.00%)

Примеры длинных контекстов:

Контекст 1:
What time is it? [SEP] I go to church mainly to keep my wife happy, but! I dont know, Ive nEver actually thought that God could rea

In [15]:
def select_best_context(row):
    """Формирует контекст, начиная с самых недавних реплик, удаляя дубли и ограничивая длину"""

    selected_context = []
    used_sentences = set()  # Хранит уникальные предложения, чтобы избежать повторов
    question_count = 0

    # Добавление контекста с конца к началу (context_5 -> context_1)
    for i in range(5, 0, -1):  # От context_5 к context_1
        ctx = row[f"context_{i}"]
        if pd.notna(ctx) and ctx not in used_sentences:
            # Извлекаются уникальные предложения
            sentences = re.split(r"(?<=[?.!])\s+", ctx)
            for sentence in sentences:
                if len(sentence.split()) > 2 and sentence not in used_sentences:
                    used_sentences.add(sentence)
                    selected_context.append(sentence)
                    if is_question(sentence):
                        question_count += 1
                    if question_count >= 4:  # Ограничение на 4 вопроса
                        break
        if question_count >= 4:
            break

    # Добавление последней реплики (previous_line) в конец
    previous = truncate_text(row["previous_line"], 25)
    selected_context.append(previous)

    # Формирование финального контекста (обрезка с начала, оставляем последние 150 слов)
    final_context = " [SEP] ".join(selected_context)
    final_context_words = final_context.split()
    if len(final_context_words) > 150:
        final_context = " ".join(final_context_words[-150:])  # Обрезка с начала

    return final_context

# Применение исправленного кода
house_df["final_context"] = house_df.apply(select_best_context, axis=1)

# Проверка статистики заново
house_df["final_context_length"] = house_df["final_context"].apply(get_text_length)
house_df["final_context_question_count"] = house_df["final_context"].apply(count_questions_in_text)

print("\nОбновленная статистика по длине контекста:")
print(house_df["final_context_length"].describe())

print("\nОбновленная статистика по количеству вопросов:")
print(house_df["final_context_question_count"].describe())

# Примеры длинных контекстов после исправлений
print("\nПримеры длинных контекстов после исправлений:")
long_contexts = house_df[house_df["final_context_length"] > 100]["final_context"].sample(5, random_state=42).tolist()
for i, context in enumerate(long_contexts, 1):
    print(f"\nКонтекст {i}:\n{context}")



Обновленная статистика по длине контекста:
count    7417.000000
mean       82.502899
std        29.096518
min        11.000000
25%        61.000000
50%        79.000000
75%       101.000000
max       150.000000
Name: final_context_length, dtype: float64

Обновленная статистика по количеству вопросов:
count    7417.000000
mean        2.866388
std         1.420995
min         0.000000
25%         2.000000
50%         3.000000
75%         4.000000
max         8.000000
Name: final_context_question_count, dtype: float64

Примеры длинных контекстов после исправлений:

Контекст 1:
And he had a full psych evaluation. [SEP] Hes not crazy. [SEP] ER also tested for steroids. [SEP] The negative test at least means steroids is less likely. [SEP] We should discuss other possibilities. [SEP] Could cause the excess hormones that could cause the rage and would elude the ER Steroid test. [SEP] Bilateral venous sampling to find the elevated GnRH. [SEP] MRI to find the pituitary damage. [SEP] Unless, of 

1. Длина контекста. Средняя длина: 82 слова, максимальная длина: 150 слов, 50% контекстов ≤ 79 слов, что соответствует хорошему балансу.
Некоторые контексты по-прежнему длинные (101-150 слов в 25% случаев).

2. Количество вопросов. Среднее количество вопросов: 2.86.
Максимум: 8 вопросов. В 75% случаев ≤ 4 вопросов, что соответствует нашему лимиту. Иногда остается больше 4 вопросов, что нужно учесть.

3. Качество контекста
Нет дублирования – одинаковые предложения не повторяются.
Контекст строится правильно – previous_line идет в конце, перед ней — логичный поток предыдущих реплик.
Фразы осмысленные, например:
sql
Copy
Edit
And youre smart, and youre funny but you are bitter. [SEP] And youre lonely, so you treat Everyone around like theyre idiots and you get away with it because of your cane. [SEP] But youre not actually getting away with it.
Это логично: сначала общая характеристика, затем последняя реплика, которая ведет к ответу персонажа.
✅ Вывод: Теперь контексты выглядят естественно и логично.

In [16]:
def clean_previous_line(text):
    """Удаляет лишние пробелы и исправляет форматирование в последней реплике"""
    if not isinstance(text, str):
        return ""
    return text.strip()

# Применение к датасету
house_df["previous_line"] = house_df["previous_line"].apply(clean_previous_line)

# Проверка примеров
print("\nПримеры очищенных `previous_line`:")
print(house_df["previous_line"].sample(5, random_state=42))


Примеры очищенных `previous_line`:
5483                 Dextromethorphan. As in cough syrup?
4556    His kidneys are fried. If he doesnt have FMF, ...
4056                Why do you care how I feel about her?
1811    I pass a farm on my way to school. And theyre ...
763                                                 What?
Name: previous_line, dtype: object


In [25]:
def extract_last_question(row):
    """Извлекает последний осмысленный вопрос из `previous_line` или `final_context`"""

    # Поиск всех предложений в `final_context`
    sentences = re.split(r"(?<=[?.!])\s+", row["final_context"])

    # Используем `previous_line`, если там есть вопрос
    if is_question(row["previous_line"]):
        question = row["previous_line"]
    else:
        # Поиск последнего вопроса в `final_context`
        question = None
        for sentence in reversed(sentences):
            if is_question(sentence):
                question = sentence.strip()
                break

    # Если `question` не заканчивается `?`, ищем альтернативный вариант
    if question and not question.strip().endswith("?"):
        for sentence in reversed(sentences):
            if is_question(sentence) and sentence.strip().endswith("?"):
                question = sentence.strip()
                break

    # Если `question` ≤ 2 слов, ищем более развернутый вариант
    if question and len(question.split()) <= 2:
        for sentence in reversed(sentences):
            if is_question(sentence) and len(sentence.split()) > 2:
                question = sentence.strip()
                break

    # Удаляем `[SEP]` в начале `question`
    if question and question.startswith("[SEP]"):
        question = question.replace("[SEP]", "").strip()

    # Если нет нормального вопроса, используем `previous_line`
    return question if question else row["previous_line"]

# Применяем функцию
house_df["question"] = house_df.apply(extract_last_question, axis=1)

# Проверяем примеры
print("\nПримеры исправленных вопросов:")
print(house_df[["previous_line", "final_context", "question"]].sample(5, random_state=42))


Примеры исправленных вопросов:
                                          previous_line  \
5483               Dextromethorphan. As in cough syrup?   
4556  His kidneys are fried. If he doesnt have FMF, ...   
4056              Why do you care how I feel about her?   
1811  I pass a farm on my way to school. And theyre ...   
763                                               What?   

                                          final_context  \
5483  Ah, what a shame. [SEP] I meant for you. [SEP]...   
4556  Most common cause of anhedonia is sParkzophren...   
4056  You need to run a kidney function test. [SEP] ...   
1811  I wanna get depo provera. [SEP] Yeah but it wo...   
763   Syndrome X could cause a stroke, but I dont kn...   

                                               question  
5483               Dextromethorphan. As in cough syrup?  
4556                  You dont know if well get better?  
4056              Why do you care how I feel about her?  
1811  I pass a farm on my 

## Финальная структура обучающего датасета

### **Описание полей**
| Колонка        | Описание  | Пример |
|---------------|----------|--------|
| **`final_context`** | Контекст до последней реплики | `"And youre smart, and youre funny but you are bitter. [SEP] And youre lonely, so you treat Everyone around like theyre idiots and you get away with it because of your cane."` |
| **`previous_line`** | Последняя реплика перед ответом (не обрезается) | `"But youre not actually getting away with it."` |
| **`question`** | Последний вопрос из `previous_line` или контекста | `"What do you mean?"` |
| **`answer`** | Реплика Доктора Хауса (правильный ответ) | `"I'm getting away with it just fine."` |
| **`label`** | 1 (если ответ правильный), 0 (если неправильный) | `1` или `0` |

### **Логика формирования `question`**
- Если в `previous_line` есть вопрос → берем его.  
- Если в `previous_line` **нет вопроса**, но есть в `final_context` →  
  - Берем **последний вопрос** в `final_context`.  
- Если в `final_context` **несколько вопросов** → берем **последний** (он самый свежий).  
- Если в `final_context` **нет вопросов**, используем `previous_line`.  

---


In [32]:
def extract_main_answer(row):
    """Выделяет главный ответ из `line`, убирая лишние вводные слова и сарказм"""

    text = row["line"]
    if not isinstance(text, str) or text.strip() == "":
        return ""

    # Если ответ короткий, оставляем его полностью
    words = text.split()
    if len(words) <= 10:
        return text.strip()

    # Разделяем на предложения
    sentences = re.split(r"(?<=[?.!])\s+", text)

    # Используем первое осмысленное предложение
    for sentence in sentences:
        if len(sentence.split()) > 3:  # Берем осмысленное предложение
            return sentence.strip()

    # Если ничего не подошло, возвращаем весь ответ
    return text.strip()

# Применяем функцию
house_df["answer"] = house_df.apply(extract_main_answer, axis=1)

# Проверяем примеры
print("\nПримеры ответов:")
print(house_df[["line", "answer"]].sample(5, random_state=42))



Примеры исправленных ответов:
                                                   line  \
5483  He wasnt taking it for his cough. Its cheap, a...   
4556                           Boy, sure hope Im right.   
4056  Because now, I know that I can get you to do a...   
1811                   Make love, not belts. Beautiful.   
763   I take it you nEver mentioned this during any ...   

                                                 answer  
5483                  He wasnt taking it for his cough.  
4556                           Boy, sure hope Im right.  
4056  Because now, I know that I can get you to do a...  
1811                              Make love, not belts.  
763   I take it you nEver mentioned this during any ...  


In [33]:
import re

def clean_answer(text):
    """Очищает `answer` от вводных слов, выбирает осмысленное предложение"""

    if not isinstance(text, str) or text.strip() == "":
        return ""

    # Удаление вводных слов
    text = re.sub(r"^(Well|Yeah|Actually|You know|Look|Listen|Right|Oh|Okay|Anyway|So),?\s+", "", text, flags=re.IGNORECASE)

    # Разделение на предложения
    sentences = re.split(r"(?<=[?.!])\s+", text)

    # Поиск осмысленного ответа (игнорируя сарказм)
    for sentence in sentences:
        if len(sentence.split()) > 3 and "yeah" not in sentence.lower() and "sure" not in sentence.lower():
            return sentence.strip()

    # Если ничего не найдено, возвращается весь `line`
    return text.strip()

# Применение функции
house_df["answer"] = house_df["line"].apply(clean_answer)

# Проверка примеров
print("\nПримеры исправленных ответов:")
print(house_df[["line", "answer"]].sample(5, random_state=42))



Примеры исправленных ответов:
                                                   line  \
5483  He wasnt taking it for his cough. Its cheap, a...   
4556                           Boy, sure hope Im right.   
4056  Because now, I know that I can get you to do a...   
1811                   Make love, not belts. Beautiful.   
763   I take it you nEver mentioned this during any ...   

                                                 answer  
5483                  He wasnt taking it for his cough.  
4556                           Boy, sure hope Im right.  
4056  Because now, I know that I can get you to do a...  
1811                              Make love, not belts.  
763   I take it you nEver mentioned this during any ...  


In [31]:
def update_question_if_duplicate(row):
    """Заменяет `question`, если он почти идентичен `previous_line`"""
    if row["cosine_question_previous"] > 0.95:  # Если `question` дублирует `previous_line`
        sentences = re.split(r"(?<=[?.!])\s+", row["final_context"])  # Разделение контекста
        for sentence in reversed(sentences):  # Берем последний вопрос в `final_context`
            if is_question(sentence) and sentence.strip() != row["previous_line"]:
                return sentence.strip()
    return row["question"]

# Применение корректировки
house_df["question"] = house_df.apply(update_question_if_duplicate, axis=1)

print("\nОбновлены `question`, если они дублировали `previous_line`.")


Обновлены `question`, если они дублировали `previous_line`.


In [27]:
def get_text_length(text):
    """Возвращает количество слов в тексте"""
    return len(text.split()) if isinstance(text, str) else 0

# Добавляем колонку с длиной ответа
house_df["answer_length"] = house_df["answer"].apply(get_text_length)

# Анализ распределения длины ответов
length_stats = house_df["answer_length"].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95])

# Вывод статистики по длине ответов
print("\nРаспределение длины ответов:")
print(length_stats)



Распределение длины ответов:
count    7417.000000
mean        8.457597
std         4.704667
min         3.000000
25%         5.000000
50%         7.000000
75%        10.000000
90%        15.000000
95%        18.000000
max        37.000000
Name: answer_length, dtype: float64


Средняя длина (mean): 8.45 слов → нормально, ответы не слишком короткие и не слишком длинные.
Максимальная длина (max): 37 слов → нет выбросов, все в разумных пределах.
75% ответов ≤ 10 слов, 95% ответов ≤ 18 слов → оптимально для кросс-энкодера.
Минимальная длина (min): 3 слова → достаточно, чтобы быть осмысленным ответом.


**Алгоритм формирования ответов Доктора Хауса (answer)**
Ответы в line не всегда:
- Релевантны – бывают отвлеченные фразы, сарказм, или отсылки.
- В начале реплики – могут идти после вводных фраз, саркастических замечаний.
- Четко соответствуют вопросу – могут быть ироничными или резкими.
Поэтому нужно выделить главную часть ответа, чтобы модель училась правильно их оценивать.

Логика формирования answer
- Оставляем line, если он короткий и осмысленный
- Если в line ≤ 10 слов, используем его полностью.
- Если line длинный (> 10 слов), выделяем главный ответ
- Ищем первое предложение (до . или ?, если это полноценный ответ).
- Если первое предложение очень короткое (≤ 3 слов), ищем следующее осмысленное предложение.
- Если в line есть сарказм, выделяем фразу, относящуюся к вопросу
- Ищем ключевые слова (yes, no, maybe, because, that's why, obviously, of course и т. д.). Если такие слова есть, оставляем часть ответа после них.
- Если ничего не подошло, оставляем line полностью


**Проверка косинусной близости в данных**

Косинусная близость помогает оценить степень схожести между текстами. Проверка проводится для того, чтобы:
- Фильтровать слабосвязанные вопросы и ответы – если ответ не имеет отношения к вопросу, его можно исключить.
- Проверять `question` и `previous_line` – если они практически идентичны, возможно, `question` не добавляет новой информации и его нужно заменить.
- Создавать более осмысленные негативные примеры (`negative_pairs`) – подбор реплик с низкой косинусной близостью позволяет формировать реалистичные отрицательные примеры.

 **Какие поля сравниваются?**
1. `question` ↔ `answer` – проверка, насколько ответ связан с вопросом.
2. `question` ↔ `previous_line` – проверка, отличается ли вопрос от последней реплики, чтобы избежать дублирования.

 **Какие значения считаются нормальными**
- `cosine_question_answer` (вопрос и ответ)  
  - ысокая близость (≥ 0.7) – ответ хорошо соответствует вопросу.  
  - Средняя близость (0.4 – 0.7) – ответ частично связан с вопросом.  
  - Низкая близость (< 0.4) – ответ может быть нерелевантным, возможно, его стоит удалить.  

- **`cosine_question_previous` (вопрос и предыдущая реплика)**  
  - Высокая близость (> 0.9) – `question` дублирует `previous_line`, стоит проверить, нужен ли этот вопрос.  
  - Средняя близость (0.5 – 0.9) – `question` логично вытекает из контекста, что является нормальным.  
  - Низкая близость (< 0.5) – `question` может быть сформулирован слишком обобщенно или не связан с предыдущей репликой.  

 **Что делать с результатами**
- Если `cosine_question_answer` слишком низкая, можно удалить эти пары.  
- Если `cosine_question_previous` слишком высокая, `question` можно заменить на более информативный.  
- Для негативных примеров (`negative_pairs`) подбираются ответы с низкой косинусной близостью к вопросу.  



In [29]:
from sentence_transformers import SentenceTransformer, util
import torch

# Загружается модель для эмбеддингов
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def encode_texts(texts):
    """Векторизует список текстов с использованием модели SentenceTransformer"""
    return model.encode(texts, convert_to_tensor=True)

# Векторизация `question`, `answer`, `previous_line`
questions_emb = encode_texts(house_df["question"].tolist())
answers_emb = encode_texts(house_df["answer"].tolist())
previous_lines_emb = encode_texts(house_df["previous_line"].tolist())

# Вычисление косинусной близости
house_df["cosine_question_answer"] = [
    util.pytorch_cos_sim(questions_emb[i], answers_emb[i]).item() for i in range(len(house_df))
]

house_df["cosine_question_previous"] = [
    util.pytorch_cos_sim(questions_emb[i], previous_lines_emb[i]).item() for i in range(len(house_df))
]

# Вывод статистики
print("\nРаспределение косинусной близости между `question` и `answer`:")
print(house_df["cosine_question_answer"].describe())

print("\nРаспределение косинусной близости между `question` и `previous_line`:")
print(house_df["cosine_question_previous"].describe())



Распределение косинусной близости между `question` и `answer`:
count    7417.000000
mean        0.207865
std         0.197324
min        -0.134349
25%         0.088306
50%         0.158947
75%         0.262798
max         1.000000
Name: cosine_question_answer, dtype: float64

Распределение косинусной близости между `question` и `previous_line`:
count    7417.000000
mean        0.765244
std         0.373661
min        -0.108959
25%         0.438442
50%         1.000000
75%         1.000000
max         1.000000
Name: cosine_question_previous, dtype: float64


Для добавления негативных реплик внесли изменения по сравнению с формированием триплетов для биэнкодера.
- Ускорити код с помощью torch.topk() вместо np.argsort().
- Считаем top_k не для всех примеров, а только для релевантных.
- Добавили фильтрацию по длине, чтобы антагонистический ответ не был намного длиннее или короче вопроса.
- Сделали пороги (0.05 < sim < 0.2) параметрами, чтобы настраивать выборку при необходимости.

In [37]:
import torch
from sentence_transformers import util

# Повторная векторизация `questions_emb` с той же моделью
questions_emb = model.encode(house_df["question"].tolist(), convert_to_tensor=True, device=device)

# Проверка размерности эмбеддингов
print("Размер `questions_emb`:", questions_emb.shape)
print("Размер `answers_emb`:", answers_emb.shape)

# Пересчет косинусной близости `question` ↔ `answer`
house_df["cosine_question_answer"] = [
    util.pytorch_cos_sim(questions_emb[i], answers_emb[i]).item() for i in range(len(house_df))
]

# Вывод обновленной статистики
print("\nОбновленное распределение косинусной близости между `question` и `answer`:")
print(house_df["cosine_question_answer"].describe())


Размер `questions_emb`: torch.Size([7417, 768])
Размер `answers_emb`: torch.Size([7417, 768])

Обновленное распределение косинусной близости между `question` и `answer`:
count    7417.000000
mean        0.188792
std         0.192068
min        -0.177711
25%         0.073482
50%         0.141968
75%         0.241475
max         1.000000
Name: cosine_question_answer, dtype: float64


In [38]:
low_similarity_examples = house_df[house_df["cosine_question_answer"] < 0.1].sample(5, random_state=42)
print("\nПримеры с низкой косинусной близостью (`question` ↔ `answer`):")
print(low_similarity_examples[["question", "answer", "cosine_question_answer"]])



Примеры с низкой косинусной близостью (`question` ↔ `answer`):
                                               question  \
558                             [SEP] Like you are now?   
6076       You think her exhusband tried to poison her?   
4932  A life where Ill continually drive away anyone...   
6589          Youre not signing that for moral reasons?   
4732    What if its a cancer syndrome, like Trousseaus?   

                                                 answer  \
558   If you do the surgery hes gonna lay on that ta...   
6076  And you would have Even it you hadnt told me t...   
4932                              Those are the breaks.   
6589        I think she looks beautiful the way she is.   
4732  And why hes gone three years without anyone se...   

      cosine_question_answer  
558                 0.058105  
6076                0.077848  
4932                0.034177  
6589                0.009933  
4732                0.032084  


In [41]:
def refine_answer(row):
    """Выбирает более релевантный ответ, если текущий имеет низкую косинусную близость"""

    if row["cosine_question_answer"] < 0.1:  # Если ответ слишком нерелевантен
        sentences = re.split(r"(?<=[?.!])\s+", row["line"])  # Разбиваем `line` на предложения

        # Исключение саркастичных фраз
        sarcasm_markers = ["never", "sure", "right", "of course", "obviously"]

        for sentence in sentences:
            if len(sentence.split()) > 3 and not any(word in sentence.lower() for word in sarcasm_markers):
                return sentence.strip()

    return row["answer"]  # Если ничего не найдено, оставляем текущий ответ

# Обновляем `answer`
house_df["answer"] = house_df.apply(refine_answer, axis=1)

# Повторная векторизация `answer`
answers_emb = model.encode(house_df["answer"].tolist(), convert_to_tensor=True, device=device)

# Повторный пересчет косинусной близости
house_df["cosine_question_answer"] = [
    util.pytorch_cos_sim(questions_emb[i], answers_emb[i]).item() for i in range(len(house_df))
]

# Проверка нового распределения
print("\nОбновленное распределение косинусной близости (`question` ↔ `answer`):")
print(house_df["cosine_question_answer"].describe())


Обновленное распределение косинусной близости (`question` ↔ `answer`):
count    7417.000000
mean        0.190341
std         0.192679
min        -0.177711
25%         0.075317
50%         0.143255
75%         0.242321
max         1.000000
Name: cosine_question_answer, dtype: float64


In [42]:
low_similarity_examples = house_df[house_df["cosine_question_answer"] < 0.1].sample(5, random_state=42)
print("\nПримеры с низкой косинусной близостью (`question` ↔ `answer`):")
print(low_similarity_examples[["question", "answer", "cosine_question_answer"]])


Примеры с низкой косинусной близостью (`question` ↔ `answer`):
                                      question  \
6523                Doing it I dont know once?   
1089  Is that what you medically need to know?   
4003           What did you get for Christmas?   
2546           [SEP] What are you looking for?   
5189                      What does it matter?   

                                                 answer  \
6523  Im not gonna act like theres a crisis before w...   
1089                   What jewelry did your bride wear   
4003                           Its like watParkng some!   
2546  Hes telling us what hes seeing, telling us exa...   
5189                                I saw guilt in him.   

      cosine_question_answer  
6523               -0.018382  
1089               -0.010579  
4003                0.055450  
2546                0.087292  
5189                0.091719  


In [43]:
def refine_answer(row):
    """Выбирает более релевантный ответ, если текущий имеет низкую косинусную близость"""

    if row["cosine_question_answer"] < 0.1:  # Если ответ слишком нерелевантен
        sentences = re.split(r"(?<=[?.!])\s+", row["line"])  # Разбиваем `line` на предложения

        # Исключение саркастичных фраз
        sarcasm_markers = ["never", "sure", "right", "of course", "obviously"]

        for sentence in sentences:
            if len(sentence.split()) > 3 and not any(word in sentence.lower() for word in sarcasm_markers):
                return sentence.strip()

    return row["answer"]  # Если ничего не найдено, оставляем текущий ответ

# Обновляем `answer`
house_df["answer"] = house_df.apply(refine_answer, axis=1)

# Повторная векторизация `answer`
answers_emb = model.encode(house_df["answer"].tolist(), convert_to_tensor=True, device=device)

# Повторный пересчет косинусной близости
house_df["cosine_question_answer"] = [
    util.pytorch_cos_sim(questions_emb[i], answers_emb[i]).item() for i in range(len(house_df))
]

# Проверка нового распределения
print("\nОбновленное распределение косинусной близости (`question` ↔ `answer`):")
print(house_df["cosine_question_answer"].describe())


Обновленное распределение косинусной близости (`question` ↔ `answer`):
count    7417.000000
mean        0.190341
std         0.192679
min        -0.177711
25%         0.075317
50%         0.143255
75%         0.242321
max         1.000000
Name: cosine_question_answer, dtype: float64


In [ ]:
import torch
import numpy as np
from sentence_transformers import util

# Параметры выбора негативных примеров
threshold_lower = 0.05   # Минимальная схожесть (слишком далекое — игнорируется)
threshold_upper = 0.2    # Максимальная схожесть (слишком похожее — игнорируется)
top_k = 50               # Берем `top_k` лучших кандидатов
max_length_diff = 10     # Максимальная разница в длине ответа и антагониста

# Векторизация `antagonists` (делается один раз!)
antagonist_lines = antagonists_df["line"].tolist()
antagonist_embeddings = model.encode(antagonist_lines, convert_to_tensor=True, device=device)

# Создание массива для негативных примеров
new_hard_negatives = np.full(len(house_df), "", dtype=object)

# Векторизация `question` и обработка по батчам
batch_size = 128  # Можно уменьшать, если GPU перегружен
questions = house_df["question"].tolist()

for i in range(0, len(questions), batch_size):
    batch_questions = questions[i:i+batch_size]  # Берем батч вопросов
    batch_embeddings = model.encode(batch_questions, convert_to_tensor=True, device=device)

    # Вычисление косинусной близости для батча
    batch_similarities = util.pytorch_cos_sim(batch_embeddings, antagonist_embeddings).cpu()

    # Обработка каждого примера в батче
    for j in range(len(batch_questions)):
        question_idx = i + j  # Индекс текущего вопроса в house_df
        similarities_row = batch_similarities[j]  # Косинусная близость для этого вопроса

        # Фильтрация по порогу схожести
        sorted_idx = torch.topk(similarities_row, k=top_k, largest=True).indices.numpy()
        candidates = [
            idx for idx in sorted_idx
            if threshold_lower < similarities_row[idx].item() < threshold_upper
        ]

        # Дополнительная фильтрация по длине ответа
        question_length = len(questions[question_idx].split())
        filtered_candidates = [
            idx for idx in candidates
            if abs(len(antagonist_lines[idx].split()) - question_length) <= max_length_diff
        ]

        # Выбор случайного негативного примера
        if len(filtered_candidates) > 0:
            neg_idx = np.random.choice(filtered_candidates)
            new_hard_negatives[question_idx] = antagonist_lines[neg_idx]

# Добавление негативных примеров в `house_df`
house_df["neg_answer"] = new_hard_negatives
print(f"Обновлено {len(new_hard_negatives)} негативных примеров.")


In [ ]:
# Создание DataFrame с позитивными примерами (`label = 1`)
positive_pairs = house_df[["final_context", "question", "answer"]].copy()
positive_pairs["label"] = 1  # Метка правильного ответа

# Создание DataFrame с негативными примерами (`label = 0`)
negative_pairs = house_df[["final_context", "question", "neg_answer"]].copy()
negative_pairs.rename(columns={"neg_answer": "answer"}, inplace=True)  # Переименовать колонку для единого формата
negative_pairs["label"] = 0  # Метка неправильного ответа

# Объединение позитивных и негативных примеров
final_dataset = pd.concat([positive_pairs, negative_pairs], ignore_index=True)

# Сохранение датасета
final_dataset.to_csv("cross_encoder_dataset.csv", index=False, encoding="utf-8")

print(f"Финальный датасет сохранен: {len(final_dataset)} строк.")
print(final_dataset.head())


In [ ]:
df = final_dataset

# Создание списка данных для кросс-энкодера
reranker_data = []

for _, row in df.iterrows():
    # Добавление позитивного примера (правильный ответ)
    reranker_data.append({"combined": f"{row['final_context']} [SEP] {row['question']} [SEP] {row['answer']}", "label": 1})

    # Добавление негативного примера (неправильный ответ)
    reranker_data.append({"combined": f"{row['final_context']} [SEP] {row['question']} [SEP] {row['neg_answer']}", "label": 0})

# Преобразование в DataFrame
reranker_df = pd.DataFrame(reranker_data)

# Сохранение в `.pkl` для обучения
reranker_df.to_pickle("reranker_dataset.pkl")

print(f"Создан датасет для кросс-энкодера: {len(reranker_df)} пар.")

